# Check Spots

Check spots for possible duplication.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of GP3S Query and is distributed under the terms of the GNU General Public License.

GP3S Query is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GP3S Query is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GP3S Query. If not, see https://www.gnu.org/licenses/.

## Import Common Modules

In [1]:
import os
import sys

import csv
import json

from datetime import datetime

from math import pi, radians, cos, sin, asin, atan2, sqrt

from lxml import etree

## Constants

Earth's [mean radius](https://en.wikipedia.org/wiki/Earth_radius#Published_values) was taken from Wikipedia - IUGG and IERS both give a value of 6371008.7714 metres.

In [2]:
EARTH_RADIUS = 6371009
EARTH_CIRCUM = 2 * pi * EARTH_RADIUS
DIST_PER_DEG_LAT = EARTH_CIRCUM / 360

PROXIMITY_THRESHOLD = 1000

## Proximity Check

Approximate distance check taken from GPS Wizard - [Haversine vs Pythagoras](https://github.com/Logiqx/gps-wizard/blob/main/python/examples/haversine_vs_pythagoras.ipynb)

In [3]:
def estimateDistanceDegrees(latitude1d, longitude1d, latitude2d, longitude2d, radius=EARTH_RADIUS):
    '''Estimate the Euclidean distance between two points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south. n.b. Can cache the values of y1 + y2
    y1 = latitude1d * DIST_PER_DEG_LAT
    y2 = latitude2d * DIST_PER_DEG_LAT
    yDelta = y2 - y1

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitude2d - longitude1d)

    # Note: This adjustment has not been factored into the "cost" comparison above because it is so unlikely to occur!
    if longDelta > 180:
        longDelta -= 360

    # Calculate distance east / west. n.b. Can even cache DIST_PER_DEG_LAT * cos(radians(latitude2d))
    xDelta = longDelta * DIST_PER_DEG_LAT * cos(radians(latitude2d))

    # Apply Pythagorean theorem to determine the distance. Distance squared can even be returned, thus avoiding sqrt()
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance

## KML Writer

Yes, I know there are libraries to do this task.

However, I already had some custom code that I have copy / pasted into this script.

In [4]:
def prepareKml(pins):
    '''Prepare KML prior to being saved'''

    # XML document
    xsi = 'http://www.w3.org/2001/XMLSchema-instance'
    kml = etree.Element(
        'kml', 
         {etree.QName(xsi, 'schemaLocation'): 'http://www.opengis.net/kml/2.2 http://schemas.opengis.net/kml/2.2.0/ogckml22.xsd'},
         nsmap={None: 'http://www.opengis.net/kml/2.2', 'xsi': xsi})

    # Document
    document = etree.SubElement(kml, 'Document')
    documentName = etree.SubElement(document, 'name')
    documentName.text = 'GP3S Spot Cluster'
    documentOpen = etree.SubElement(document, 'open')
    documentOpen.text = '1'

    # Style
    style = etree.SubElement(document, 'Style')
    style.attrib['id'] = 'gp3s'

    # Each track will be a new placemark
    for pin in pins:
        # Placemark
        placemark = etree.SubElement(document, 'Placemark')
        placemarkName = etree.SubElement(placemark, 'name')
        placemarkName.text = pin['name']

        # Style
        placemarkStyleUrl = etree.SubElement(placemark, 'styleUrl')
        placemarkStyleUrl.text = '#gp3s'

        # Point
        point = etree.SubElement(placemark, 'Point')

        # Coordinate
        coordinates = etree.SubElement(point, 'coordinates')
        coordinates.text = '{},{}'.format(pin['lon'], pin['lat'])

    return etree.tostring(kml, pretty_print=True, xml_declaration=False, encoding='UTF-8',
                                 doctype='<?xml version="1.0" encoding="UTF-8"?>')

## Check the Spots

Check the spots.

Fields:
- SPOT_TYPE (Lake, Sea, Speed Canal)
- GoogleEarth_Link (uses names)
- Class_ID (-3, -1, 23, 24, 25, 26) is related to SPOT_TPYE
- date_add is not very useful because some dates are very common (batch add)
- NRSessions

Integrity:
- Check for spaces in spot name (appear at top of list)
- Spots without country ID - 1540 (Sacca di Goro)

Merging:
- What about peoples spots list?
- What about home spots?
- What about achievements, spot records, PRs, etc?

In [5]:
FIELD_SPOT_ID = 'spot_id'
FIELD_SPOT_NAME = 'Spot_Name'
FIELD_SPOT_TYPE = 'SPOT_TYPE'

FIELD_COUNTRY_ID = 'country_id'
FIELD_COUNTRY_NAME = 'SPOT_COUNTRY'

FIELD_LON = 'Long'
FIELD_LAT = 'Lat'

FIELD_NUM_SESSIONS = 'NRSessions'

SPOT_SEARCH_URL = 'https://www.gps-speedsurfing.com/mygps.aspx?mnu=spotsearch&val='

GOOGLE_MAPS_URL = 'https://www.google.com/maps/search/?api=1&query='

In [6]:
def loadSpots():
    """Load spots from disk"""
    
    spotsFile = os.path.join(projdir, 'data', 'spots.csv')
    
    spots = {}

    with open(spotsFile) as f:
        reader = csv.DictReader(f)
    
        for spot in reader:
            spotId = int(spot[FIELD_SPOT_ID])

            spots[spotId] = spot
            
    return spots


def getCountries(spots):
    """Determine spots for each country"""
    
    countries = {}

    for spotId, spot in spots.items():
        if spot[FIELD_COUNTRY_ID] != 'NULL':
            countryId = int(spot[FIELD_COUNTRY_ID])

            if countryId not in countries:
                country = {'id': countryId, 'name': spot[FIELD_COUNTRY_NAME], 'spots': {}}
                countries[countryId] = country

            countries[countryId]['spots'][spotId] = spot

        else:
            print('Warning: No country ID for {} ({})'.format(spot[FIELD_SPOT_NAME], spotId))
        
    return countries


def getCandidates(country):
    """Check spots in a single country for possible duplicates"""
    
    candidates = []
    
    spots = country['spots']
    spotIds = sorted([spotId for spotId in spots.keys()], reverse=True)

    for i in range(len(spotIds)):
        spot1 = spots[spotIds[i]]

        if spot1[FIELD_LAT] != 'NULL':
            lat1 = float(spot1[FIELD_LAT])
            lon1 = float(spot1[FIELD_LON])

            for j in range(i + 1, len(spotIds)):
                spot2 = spots[spotIds[j]]

                if spot2[FIELD_LAT] != 'NULL':
                    lat2 = float(spot2[FIELD_LAT])
                    lon2 = float(spot2[FIELD_LON])

                    proximity = estimateDistanceDegrees(lat1, lon1, lat2, lon2)
                    
                    if proximity < PROXIMITY_THRESHOLD:
                        candidates.append((int(spot1[FIELD_SPOT_ID]), int(spot2[FIELD_SPOT_ID])))
                        
    return candidates


def getClusters(candidates):
    """Determine clusters of candidates"""

    spots = {}
    clusters = {}
    nextClusterId = 0

    for candidate in candidates:
        spotId1, spotId2 = candidate
        
        if spotId1 not in spots:
            if spotId2 not in spots:
                # Both spots are new so create a new cluster for them both
                clusterId = nextClusterId

                clusters[clusterId] = set()
                clusters[clusterId].add(spotId1)
                clusters[clusterId].add(spotId2)

                spots[spotId1] = clusterId
                spots[spotId2] = clusterId
                
                nextClusterId += 1

            else:
                # Only Spot 1 is new so add it to the same cluster as spot 2
                clusterId = spots[spotId2]

                clusters[clusterId].add(spotId1)

                spots[spotId1] = clusterId

        else:
            if spotId2 not in spots:
                # Only Spot 2 is new so add it to the same cluster as spot 1
                clusterId = spots[spotId1]

                clusters[clusterId].add(spotId2)

                spots[spotId2] = clusterId

            else:
                # Neither spot is new so combine the clusters if they are different
                clusterId1 = spots[spotId1]
                clusterId2 = spots[spotId2]
                
                # Important to recognise if they are already in the same cluster
                if clusterId1 != clusterId2:
                    cluster1 = clusters[clusterId1]
                    cluster2 = clusters[clusterId2]

                    for spotId in cluster2:
                        cluster1.add(spotId)
                        spots[spotId] = clusterId1

                    del clusters[clusterId2]
                
    return clusters


def reportClusters(country):
    """Report clusters within a single country"""

    countryId = country['id']
    countryName = country['name']

    filename = os.path.join(projdir, 'docs', 'spot-clusters', str(countryId) + '.md')  
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    with open(filename, "w") as f:
        f.write('## {} - ID {}\n\n'.format(countryName, countryId))
        
        if len(country['clusters']):
            for clusterId, cluster in country['clusters'].items():
                f.write('### Cluster {} - [KML]({}/{}.kml)\n\n'.format(clusterId + 1, countryId, clusterId + 1))
                
                f.write('| Spot Name | Spot ID | Spot Type | Num Sessions | Location |\n')
                f.write('| --------- | :-----: | :-------: | :----------: | :------: |\n')
                
                points = []

                for spotId in sorted(cluster):
                    spot = country['spots'][spotId]
                    f.write('| [{}]({}{}.md) | {} | {} | {}| [Google]({}{},{})\n'.format(
                        spot[FIELD_SPOT_NAME], SPOT_SEARCH_URL, spot[FIELD_SPOT_ID], spot[FIELD_SPOT_ID],
                        spot[FIELD_SPOT_TYPE], spot[FIELD_NUM_SESSIONS],
                        GOOGLE_MAPS_URL, spot[FIELD_LAT], spot[FIELD_LON]))
                    
                    point = {
                        'name': spot[FIELD_SPOT_NAME],
                        'lat': spot[FIELD_LAT],
                        'lon': spot[FIELD_LON]
                    }
                    
                    points.append(point)
                    
                f.write('\n')

                kml = prepareKml(points)
                
                filename = os.path.join(projdir, 'docs', 'spot-clusters', str(countryId), str(clusterId + 1) + '.kml')  
                dirname = os.path.dirname(filename)
                if not os.path.exists(dirname):
                    os.makedirs(dirname)

                with open(filename, "wb") as f2:
                    f2.write(kml)

        else:
            f.write('No spot clusters found.')


if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], '..'))
    
    spots = loadSpots()

    countries = getCountries(spots)
    
    numSpots = 0
    numClusters = 0

    filename = os.path.join(projdir, 'docs', 'spot-clusters', 'README.md')  
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    with open(filename, "w") as f:
        f.write('## GP3S Spot Clusters\n\n')

        utcnow = str(datetime.utcnow()).split('.')[0]
        f.write('Summary of spot clusters detected in the GP3S countries.\n\n')
        f.write('Last refreshed {} UTC.\n\n'.format(utcnow))

        f.write('| Country Name | Country ID | Spots | Spot Clusters |\n')
        f.write('| ------------ | :--------: | :---: | :-----------: |\n')

        summary = []
        for countryId, country in countries.items():
            spots = country['spots']

            candidates = getCandidates(country)
            clusters = getClusters(candidates)

            country['clusters'] = clusters
            reportClusters(country)

            summary.append('| [{}]({}.md) | {} | {} | {} |\n'.format(
                country['name'], countryId, countryId, len(spots.keys()), len(clusters)))

            numSpots += len(spots.keys())
            numClusters += len(clusters)
        
        for text in sorted(summary):
            f.write(text)
        f.write('| TOTAL | - | {} | {} |\n'.format(numSpots, numClusters))

    print('\n{} spots, {} spot clusters\n'.format(numSpots, numClusters))

    print('All done!')


3125 spots, 153 spot clusters

All done!
